In [2]:
// Blitz bucket data: [upper rating bound, number of players in that bucket]
const distributionData = [
  [100, 1120380],
  [200, 1640484],
  [300, 1666596],
  [400, 1592844],
  [500, 1459367],
  [600, 1284387],
  [700, 1093492],
  [800, 917629],
  [900, 747637],
  [1000, 627240],
  [1100, 510845],
  [1200, 426427],
  [1300, 331625],
  [1400, 258851],
  [1500, 202190],
  [1600, 157355],
  [1700, 118897],
  [1800, 92182],
  [1900, 67939],
  [2000, 53380],
  [2100, 37586],
  [2200, 27309],
  [2300, 18608],
  [2400, 12439],
  [2500, 10745],
  [2600, 5734],
  [2700, 3071],
  [2800, 1849],
  [2900, 807],
  [3000, 366],
];

// Official total players (from Chess.com)
const totalPlayers = 23088905;

// Extract ratings and counts
const ratings = distributionData.map(([rating]) => rating);
const rawCounts = distributionData.map(([_, count]) => count);

// Step 1: Scale counts to match official total player count
const rawTotal = rawCounts.reduce((a, b) => a + b, 0);
const scalingFactor = totalPlayers / rawTotal;
const scaledCounts = rawCounts.map(count => count * scalingFactor);

// Step 2: Create cumulative scaled counts
const cumulativeCounts = scaledCounts.reduce((acc, val, i) => {
  acc.push((acc[i - 1] || 0) + val);
  return acc;
}, []);

// Step 3: Interpolate to get percentile and rank
function estimateEmpiricalPercentileAndRank(rating) {
  if (rating < ratings[0]) {
    return { rank: totalPlayers, percentile: 0.0 };
  }

  for (let i = 1; i < ratings.length; i++) {
    const r0 = ratings[i - 1];
    const r1 = ratings[i];
    const c0 = cumulativeCounts[i - 1];
    const c1 = cumulativeCounts[i];

    if (rating >= r0 && rating < r1) {
      const ratio = (rating - r0) / (r1 - r0);
      const estCum = c0 + ratio * (c1 - c0);
      const percentile = (estCum / totalPlayers) * 100;
      const rank = totalPlayers - estCum + 1;
      return {
        rank: Math.round(rank),
        percentile: parseFloat(percentile.toFixed(2))
      };
    }
  }

  return { rank: 1, percentile: 100.0 }; // For ratings >= 3000
}

// Example test cases
const testRatings = [600, 1200, 1450, 1534, 1700, 2000, 2500, 2800, 1685];

testRatings.forEach(rating => {
  const { rank, percentile } = estimateEmpiricalPercentileAndRank(rating);
  console.log(`Rating: ${rating} -> Rank: ${rank.toLocaleString()}, Percentile: ${percentile}%`);
});

Rating: 600 -> Rank: 9,122,254, Percentile: 60.49%
Rating: 1200 -> Rank: 2,232,568, Percentile: 90.33%
Rating: 1450 -> Rank: 1,130,460, Percentile: 95.1%
Rating: 1534 -> Rank: 884,092, Percentile: 96.17%
Rating: 1700 -> Rank: 529,110, Percentile: 97.71%
Rating: 2000 -> Rank: 188,868, Percentile: 99.18%
Rating: 2500 -> Rank: 18,849, Percentile: 99.92%
Rating: 2800 -> Rank: 1,870, Percentile: 99.99%
Rating: 1685 -> Rank: 557,531, Percentile: 97.59%
